Задание: Создать программу, которая загружает данные из CSV-файла (можно скачать из открытых источников) обрабатывает их и выводит статистику, используя только стандартные возможности Python.
Входные данные:   CSV-файл из открытых источников (например, Kaggle, UCI Repository). Файл может содержать любые данные (например, погода, товары), но должен иметь минимум 4 колонки (числа, строки, категории).
Программа должна считать CSV-файл.
Данные из файла нужно сохранить с помощью одной из структур Python (словарь, список и т.д.) или их комбинации. 
Реализовать функции:
Подсчет уникальных значений в одной из колонок файла.
Фильтрация данных по числовому значению выше или меньше заданного порога.
Функция сортировки данных по любой колонке, сортировку реализовать самостоятельно.
Поиск по значению в одной из строковых колонок.
Предусмотреть обработку исключительных ситуаций.
Программа должна быть написана в объектно-ориентированном стиле.
Результаты выводить в консоль.



In [9]:
# Импорт модуля для работы с CSV файлами - чтение и запись данных в формате CSV
import csv
# Импорт модуля для работы с операционной системой - проверка путей, файлов и директорий
import os
# Импорт функции для красивого форматирования таблиц в консоли
from tabulate import tabulate

In [12]:
# Определение класса для анализа данных об экранном времени
class ScreenTimeAnalyzer:
    # Множество обязательных колонок, которые должны присутствовать в CSV файле
    REQUIRED_COLUMNS = {
        'screen_time_hours',        # Количество часов, проведенных перед экраном (числовая колонка)
        'stress_level_0_10',        # Уровень стресса по шкале от 0 до 10 (числовая колонка)
        'mental_wellness_index_0_100',  # Индекс психического благополучия по шкале от 0 до 100 (числовая колонка)
        'occupation',               # Род занятий (строковая колонка)
        'age',                      # Возраст (числовая колонка)
        'gender'                    # Пол (категориальная колонка)
    }

    # Конструктор класса, инициализирует объект при создании экземпляра класса
    def __init__(self, file_path):
        # Сохранение пути к файлу данных как атрибута объекта
        self.file_path = file_path
        # Инициализация пустого списка для хранения данных из CSV файла
        self.data = []
        # Инициализация пустого списка для хранения заголовков колонок
        self.headers = []
        # Вызов метода загрузки данных при создании экземпляра класса
        self.load_data()

    # Метод для загрузки данных из CSV файла и преобразования типов данных
    def load_data(self):
        """Загружает данные из CSV-файла и преобразует типы."""
        # Проверка существования файла по указанному пути
        if not os.path.exists(self.file_path):
            # Вызов исключения если файл не найден
            raise FileNotFoundError(f"Файл {self.file_path} не найден.")

        # Блок try-except для обработки возможных ошибок при чтении файла
        try:
            # Открытие файла в режиме чтения с указанием кодировки UTF-8
            with open(self.file_path, mode='r', encoding='utf-8') as file:
                # Создание объекта для чтения CSV файла
                reader = csv.reader(file)
                # Чтение всех строк из файла в список
                rows = list(reader)
                # Проверка что файл не пустой
                if not rows:
                    # Вызов исключения если файл пустой
                    raise ValueError("Файл пустой")

                # Обработка заголовков: удаление пробелов и пустых значений
                self.headers = [h.strip() for h in rows[0] if h.strip() != '']

                # Проверка наличия всех обязательных колонок в файле
                missing_cols = self.REQUIRED_COLUMNS - set(self.headers)
                # Если есть отсутствующие колонки
                if missing_cols:
                    # Вызов исключения с перечислением отсутствующих колонок
                    raise ValueError(f"В файле отсутствуют обязательные колонки: {', '.join(missing_cols)}")

                # Обработка строк данных (начиная со второй строки)
                for i, row in enumerate(rows[1:], start=2):
                    # Проверка что количество колонок в строке соответствует заголовкам
                    if len(row) != len(rows[0]):
                        # Вызов исключения если количество колонок не совпадает
                        raise ValueError(f"Несоответствие колонок в строке {i}.")

                    # Создание словаря для хранения данных текущей строки
                    row_dict = {}
                    # Перебор всех колонок в строке
                    for j, key in enumerate(self.headers):
                        # Получение значения из текущей колонки
                        value = row[j]
                        # Попытка преобразовать значение в число
                        try:
                            # Если значение содержит точку - преобразовать во float
                            if '.' in value:
                                row_dict[key] = float(value)
                            else:
                                # Иначе преобразовать в int
                                row_dict[key] = int(value)
                        except (ValueError, TypeError):
                            # Если преобразование не удалось - оставить строку, убрав лишние пробелы
                            row_dict[key] = value.strip() if isinstance(value, str) else value

                    # Добавление словаря с данными строки в общий список данных
                    self.data.append(row_dict)
                
                # Вывод информации о успешно загруженных данных
                print("=" * 60)
                print("ЗАДАНИЕ 1: ЗАГРУЗКА ДАННЫХ ИЗ CSV-ФАЙЛА")
                print("=" * 60)
                print(f"Данные успешно загружены. Записей: {len(self.data)}")
                print(f"Колонки: {', '.join(self.headers)}")
                print("=" * 60)

        # Обработка ошибки кодировки файла
        except UnicodeDecodeError:
            raise ValueError("Ошибка кодировки файла. Файл должен быть в UTF-8.")
        # Обработка всех остальных исключений
        except Exception as e:
            raise RuntimeError(f"Ошибка при чтении файла: {e}")

    # Метод для подсчета уникальных значений в указанной колонке
    def count_unique_values(self, column_name):
        """Подсчитывает уникальные значения в колонке и выводит результат"""
        # Проверка что указанная колонка существует в данных
        if column_name not in self.headers:
            raise ValueError(f"Колонка '{column_name}' не найдена.")

        # Создание множества для хранения уникальных значений
        unique_values = set()
        # Перебор всех строк данных
        for row in self.data:
            # Добавление значения из указанной колонки в множество
            unique_values.add(row[column_name])
        
        # Вывод результатов подсчета уникальных значений
        print("\n" + "=" * 60)
        print("ЗАДАНИЕ 1: ПОДСЧЕТ УНИКАЛЬНЫХ ЗНАЧЕНИЙ В КОЛОНКЕ")
        print("=" * 60)
        print(f"Колонка: '{column_name}'")
        print(f"Количество уникальных значений: {len(unique_values)}")
        print(f"Значения: {', '.join(map(str, unique_values))}")
        print("=" * 60)
        
        # Возврат количества уникальных значений и самого множества значений
        return len(unique_values), unique_values

    # Метод для фильтрации данных по числовому порогу
    def filter_by_threshold(self, column_name, threshold, operator='greater'):
        """Фильтрация по числовому порогу: больше или меньше с выводом результатов"""
        # Проверка что указанная колонка существует в данных
        if column_name not in self.headers:
            raise ValueError(f"Колонка '{column_name}' не найдена.")

        # Попытка преобразовать пороговое значение в число
        try:
            threshold = float(threshold)
        except ValueError:
            raise ValueError("Порог должен быть числом.")

        # Создание списка для отфильтрованных данных
        filtered = []
        # Перебор всех строк данных
        for row in self.data:
            # Получение значения из указанной колонки
            value = row[column_name]
            # Пропуск строк если значение не числовое
            if not isinstance(value, (int, float)):
                continue

            # Применение фильтра в зависимости от оператора
            if operator == 'greater' and value > threshold:
                filtered.append(row)
            elif operator == 'less' and value < threshold:
                filtered.append(row)

        # Формирование текста для вывода в зависимости от оператора
        operator_text = "больше" if operator == 'greater' else "меньше"
        # Вывод результатов фильтрации
        print("\n" + "=" * 60)
        print("ЗАДАНИЕ 1: ФИЛЬТРАЦИЯ ДАННЫХ ПО ЧИСЛОВОМУ ПОРОГУ")
        print("=" * 60)
        print(f"Колонка: '{column_name}'")
        print(f"Условие: {operator_text} {threshold}")
        print(f"Найдено записей: {len(filtered)}")
        # Вывод отфильтрованных данных в виде таблицы
        self.display_data(filtered, limit=5)
        print("=" * 60)
        
        # Возврат отфильтрованных данных
        return filtered

    # Метод для сортировки данных пузырьковым методом по указанной колонке
    def bubble_sort_by_column(self, column_name, reverse=False):
        """Сортировка пузырьком по указанной колонке с выводом результатов"""
        # Проверка что указанная колонка существует в данных
        if column_name not in self.headers:
            raise ValueError(f"Колонка '{column_name}' не найдена.")

        # Создание копии данных для сортировки (чтобы не изменять оригинальные данные)
        data_copy = self.data[:]
        # Получение количества элементов в данных
        n = len(data_copy)
        
        # Реализация алгоритма пузырьковой сортировки
        for i in range(n):
            for j in range(0, n - i - 1):
                # Получение значений из текущей и следующей строки для сравнения
                val1 = data_copy[j][column_name]
                val2 = data_copy[j + 1][column_name]

                # Попытка преобразовать значения к числам для корректного сравнения
                try:
                    val1 = float(val1)
                    val2 = float(val2)
                except (ValueError, TypeError):
                    # Если преобразование не удалось - оставить значения как есть (для строк)
                    pass

                # Определение необходимости обмена элементов местами
                should_swap = False
                if reverse:
                    # Для сортировки по убыванию
                    if val1 < val2:
                        should_swap = True
                else:
                    # Для сортировки по возрастанию
                    if val1 > val2:
                        should_swap = True

                # Обмен элементов местами если необходимо
                if should_swap:
                    data_copy[j], data_copy[j + 1] = data_copy[j + 1], data_copy[j]

        # Формирование текста для вывода в зависимости от направления сортировки
        order_text = "по убыванию" if reverse else "по возрастанию"
        # Вывод результатов сортировки
        print("\n" + "=" * 60)
        print("ЗАДАНИЕ 1: СОРТИРОВКА ДАННЫХ (ПУЗЫРЬКОВАЯ СОРТИРОВКА)")
        print("=" * 60)
        print(f"Колонка: '{column_name}'")
        print(f"Порядок: {order_text}")
        # Вывод отсортированных данных в виде таблицы
        self.display_data(data_copy, limit=5)
        print("=" * 60)
        
        # Возврат отсортированных данных
        return data_copy

    # Метод для поиска по подстроке в строковой колонке
    def search_by_substring(self, column_name, substring):
        """Поиск по подстроке в строковой колонке с выводом результатов"""
        # Проверка что указанная колонка существует в данных
        if column_name not in self.headers:
            raise ValueError(f"Колонка '{column_name}' не найдена.")

        # Создание списка для хранения результатов поиска
        results = []
        # Перебор всех строк данных
        for row in self.data:
            # Приведение значения к строке и нижнему регистру для регистронезависимого поиска
            value = str(row[column_name]).lower()
            # Проверка наличия подстроки в значении
            if substring.lower() in value:
                # Добавление строки в результаты если подстрока найдена
                results.append(row)

        # Вывод результатов поиска
        print("\n" + "=" * 60)
        print("ЗАДАНИЕ 1: ПОИСК ПО ПОДСТРОКЕ В СТРОКОВОЙ КОЛОНКЕ")
        print("=" * 60)
        print(f"Колонка: '{column_name}'")
        print(f"Искомая подстрока: '{substring}'")
        print(f"Найдено записей: {len(results)}")
        # Вывод найденных данных в виде таблицы
        self.display_data(results, limit=5)
        print("=" * 60)
        
        # Возврат результатов поиска
        return results

    # Метод для вывода данных в консоль в виде таблицы
    def display_data(self, data_list, limit=None):
        """Вывод данных только по REQUIRED_COLUMNS в читаемом виде"""
        if not data_list:
            print("Нет данных для отображения.")
            return
    
        # Используем только те колонки, что указаны в REQUIRED_COLUMNS (и есть в данных)
        display_columns = [col for col in self.REQUIRED_COLUMNS if col in self.headers]
    
        # Сортируем колонки для стабильного порядка (например: age, gender, occupation...)
        display_columns.sort()
    
        table_data = []
        for row in data_list:
            row_vals = []
            for col in display_columns:
                val = row.get(col, "")
                # Убираем переносы и лишние пробелы
                if isinstance(val, str):
                    val = val.replace('\n', ' ').replace('\r', '').strip()
                row_vals.append(val)
            table_data.append(row_vals)
    
        if limit and len(table_data) > limit:
            table_data = table_data[:limit]
            print(f"\nОтображаются первые {limit} строк:\n")
    
        print(tabulate(
            table_data,
            headers=display_columns,
            tablefmt="simple_outline",  # или "grid", но simple_outline компактнее
            maxcolwidths=12,
            stralign="left"
        ))

    # Метод для показа первых n строк данных
    def head(self, n=5):
        """Показывает первые n строк датасета."""
        # Вывод заголовка для раздела
        print("\n" + "=" * 60)
        print("ЗАДАНИЕ 1: ПЕРВЫЕ N СТРОК ДАТАСЕТА")
        print("=" * 60)
        print(f"Первые {n} строк датасета:")
        # Вывод первых n строк данных
        self.display_data(self.data, limit=n)
        print("=" * 60)

In [13]:
if __name__ == "__main__":
    try:
        # Создание экземпляра анализатора с указанием пути к файлу данных
        analyzer = ScreenTimeAnalyzer("ScreenTime vs MentalWellness.csv")
               
        # Показ первых 5 строк данных
        analyzer.head(5)
        
        # Подсчет уникальных значений в колонке 'gender'
        analyzer.count_unique_values('gender')
        
        # Фильтрация: люди с экраном > 6 часов в день
        analyzer.filter_by_threshold('screen_time_hours', 6, 'greater')
        
        # Сортировка по стрессу (по возрастанию)
        analyzer.bubble_sort_by_column('stress_level_0_10', reverse=False)
        
        # Поиск: студенты (student)
        analyzer.search_by_substring('occupation', 'student')
        
    # Обработка исключений при возникновении ошибок
    except Exception as e:
        print(f"Ошибка: {e}")

ЗАДАНИЕ 1: ЗАГРУЗКА ДАННЫХ ИЗ CSV-ФАЙЛА
Данные успешно загружены. Записей: 400
Колонки: user_id, age, gender, occupation, work_mode, screen_time_hours, work_screen_hours, leisure_screen_hours, sleep_hours, sleep_quality_1_5, stress_level_0_10, productivity_0_100, exercise_minutes_per_week, social_hours_per_week, mental_wellness_index_0_100

ЗАДАНИЕ 1: ПЕРВЫЕ N СТРОК ДАТАСЕТА
Первые 5 строк датасета:

Отображаются первые 5 строк:

┌───────┬──────────┬───────────────────────────────┬──────────────┬─────────────────────┬─────────────────────┐
│   age │ gender   │   mental_wellness_index_0_100 │ occupation   │   screen_time_hours │   stress_level_0_10 │
├───────┼──────────┼───────────────────────────────┼──────────────┼─────────────────────┼─────────────────────┤
│    33 │ Female   │                           9.3 │ Employed     │               10.79 │                 9.3 │
│    28 │ Female   │                          56.2 │ Employed     │                7.4  │                 5.7 │
│    3